# Langchain Basics

In [61]:
%%time
!pip install openai langchain langchain_community langchain_experimental langchain_openai pinecone-client python-dotenv tiktoken -Uq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.2/203.2 kB 1.7 MB/s eta 0:00:00
CPU times: user 13.2 ms, sys: 8.1 ms, total: 21.3 ms
Wall time: 2.81 s


In [2]:
!pip show openai langchain pinecone-client python-dotenv tiktoken

Name: openai
Version: 1.37.1
Summary: The official Python library for the openai API
Home-page: https://github.com/openai/openai-python
Author: 
Author-email: OpenAI <support@openai.com>
License: 
Location: /usr/local/lib/python3.10/dist-packages
Requires: anyio, distro, httpx, pydantic, sniffio, tqdm, typing-extensions
Required-by: langchain-openai
---
Name: langchain
Version: 0.2.11
Summary: Building applications with LLMs through composability
Home-page: https://github.com/langchain-ai/langchain
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.10/dist-packages
Requires: aiohttp, async-timeout, langchain-core, langchain-text-splitters, langsmith, numpy, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: langchain-community
---
Name: pinecone-client
Version: 5.0.0
Summary: Pinecone client and SDK
Home-page: https://www.pinecone.io
Author: Pinecone Systems, Inc.
Author-email: support@pinecone.io
License: Apache-2.0
Location: /usr/local/lib/python3.10/dis

In [3]:
!pip install langchain_community -q

## Python Dot-Env

In [4]:
# vscode
import os
from dotenv import load_dotenv, find_dotenv

load_dotenv(dotenv_path=find_dotenv(), override=True)

OPENAI_API_KEY=os.getenv(key="OPENAI_API_KEY")
PINECONE_API_KEY=os.getenv(key="PINECONE_API_KEY")
LANGCHAIN_API_KEY=os.getenv(key="LANGCHAIN_API_KEY")

In [5]:
# colab
from google.colab import userdata

OPENAI_API_KEY=userdata.get('openai_key')
PINECONE_API_KEY=userdata.get('PINECONE_API_KEY')
LANGCHAIN_API_KEY=userdata.get('LANGCHAIN_API_KEY')

## Simple LLM → `gpt-3.5-turbo-instruct`

In [6]:
from langchain.llms.openai import OpenAI

llm=OpenAI(
    model_name="gpt-3.5-turbo-instruct",
    openai_api_key=OPENAI_API_KEY,
    temperature=0.9,
    max_tokens=512
  )
print(llm)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


OpenAI
Params: {'model_name': 'gpt-3.5-turbo-instruct', 'temperature': 0.9, 'top_p': 1.0, 'frequency_penalty': 0.0, 'presence_penalty': 0.0, 'n': 1, 'logit_bias': {}, 'max_tokens': 512}


In [7]:
prompt="Explain LLM Wrappers and it's use-cases in three sentences."
output=llm(prompt)
print(output)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




1. LLM (Local Linear Mapping) Wrappers are machine learning techniques used for dimensionality reduction in high-dimensional data.
2. These methods aim to preserve the local structure of the data by finding a low-dimensional representation that maximally preserves the pairwise distances among the data points.
3. LLM Wrappers are particularly useful in data visualization, feature selection, and clustering tasks where high-dimensional data can be more easily understood and analyzed in a lower-dimensional space.


In [8]:
prompt_token_length=llm.get_num_tokens(prompt)
print(f"Prmopt token length: {prompt_token_length}")

Prmopt token length: 16


In [9]:
# generate multiple prompts
prompt1="... is the city of joy."
prompt2="Top 10 richest persons are ..."
output=llm.generate([prompt1, prompt2])
print(output)

generations=[[Generation(text=" A thousand different influences contribute to the amazing energy and vibrancy which make it unique. From innumerable palaces and museums to the never-ending festivals Kolkata is a city that is a feast to the eyes. The city has been home to luminaries like Rabindranath Tagore, Satyajit Ray, Mother Teresa, Subhas Chandra Bose and Netaji by name. Kolkata is also home to India's film industry- Bollywood. Its people are enthusiastic, expressive, moody, musical and artistic-by human...\n\n754 Words | 2 Pages\n\nOpen Document\n\nWhy Our Veterans Should Be Honored\n\nSo why should we honor our veterans? I think we should honor our veterans for many reasons and many ways. I will... explain my reasons in this essay. The first reason why we should honor veterans is because they fought for our freedom. You may ask, what is freedom? Freedom is the power or right to act, speak, or think as one wants without hindrance or restraint. My grandfather, like many veterans, f

In [10]:
print(len(output.generations), output.generations)

2 [[Generation(text=" A thousand different influences contribute to the amazing energy and vibrancy which make it unique. From innumerable palaces and museums to the never-ending festivals Kolkata is a city that is a feast to the eyes. The city has been home to luminaries like Rabindranath Tagore, Satyajit Ray, Mother Teresa, Subhas Chandra Bose and Netaji by name. Kolkata is also home to India's film industry- Bollywood. Its people are enthusiastic, expressive, moody, musical and artistic-by human...\n\n754 Words | 2 Pages\n\nOpen Document\n\nWhy Our Veterans Should Be Honored\n\nSo why should we honor our veterans? I think we should honor our veterans for many reasons and many ways. I will... explain my reasons in this essay. The first reason why we should honor veterans is because they fought for our freedom. You may ask, what is freedom? Freedom is the power or right to act, speak, or think as one wants without hindrance or restraint. My grandfather, like many veterans, fought for 

In [11]:
print(output.generations[0][0])

text=" A thousand different influences contribute to the amazing energy and vibrancy which make it unique. From innumerable palaces and museums to the never-ending festivals Kolkata is a city that is a feast to the eyes. The city has been home to luminaries like Rabindranath Tagore, Satyajit Ray, Mother Teresa, Subhas Chandra Bose and Netaji by name. Kolkata is also home to India's film industry- Bollywood. Its people are enthusiastic, expressive, moody, musical and artistic-by human...\n\n754 Words | 2 Pages\n\nOpen Document\n\nWhy Our Veterans Should Be Honored\n\nSo why should we honor our veterans? I think we should honor our veterans for many reasons and many ways. I will... explain my reasons in this essay. The first reason why we should honor veterans is because they fought for our freedom. You may ask, what is freedom? Freedom is the power or right to act, speak, or think as one wants without hindrance or restraint. My grandfather, like many veterans, fought for freedom. He fou

In [12]:
# access only the first prompt result
print(output.generations[0][0].text)

 A thousand different influences contribute to the amazing energy and vibrancy which make it unique. From innumerable palaces and museums to the never-ending festivals Kolkata is a city that is a feast to the eyes. The city has been home to luminaries like Rabindranath Tagore, Satyajit Ray, Mother Teresa, Subhas Chandra Bose and Netaji by name. Kolkata is also home to India's film industry- Bollywood. Its people are enthusiastic, expressive, moody, musical and artistic-by human...

754 Words | 2 Pages

Open Document

Why Our Veterans Should Be Honored

So why should we honor our veterans? I think we should honor our veterans for many reasons and many ways. I will... explain my reasons in this essay. The first reason why we should honor veterans is because they fought for our freedom. You may ask, what is freedom? Freedom is the power or right to act, speak, or think as one wants without hindrance or restraint. My grandfather, like many veterans, fought for freedom. He fought for freedo

### Generate Multiple Responses for Single Prompt

In [13]:
# asked llm to generate multiple responses
prompt="Write tagline for a software company that does outsourcing."
outputs=llm.generate([prompt], n=2)
print(outputs)

generations=[[Generation(text='\n"Empowering your business, one outsourced solution at a time."', generation_info={'finish_reason': 'stop', 'logprobs': None}), Generation(text='\n"Empowering your business through custom solutions – Let us handle the rest."', generation_info={'finish_reason': 'stop', 'logprobs': None})]] llm_output={'token_usage': {'total_tokens': 42, 'completion_tokens': 31, 'prompt_tokens': 11}, 'model_name': 'gpt-3.5-turbo-instruct'} run=[RunInfo(run_id=UUID('064c4051-4c8a-4fe3-83b5-d4e973e826c1'))]


In [14]:
print(len(outputs.generations[0]), outputs.generations)

2 [[Generation(text='\n"Empowering your business, one outsourced solution at a time."', generation_info={'finish_reason': 'stop', 'logprobs': None}), Generation(text='\n"Empowering your business through custom solutions – Let us handle the rest."', generation_info={'finish_reason': 'stop', 'logprobs': None})]]


In [15]:
for index,output in enumerate(outputs.generations[0]):
  print(output.text)


"Empowering your business, one outsourced solution at a time."

"Empowering your business through custom solutions – Let us handle the rest."


### Generate Multiple Responses from Multiple Prompts

In [16]:
# asked llm to generate multiple responses
prompt1="Write tagline for an AI Company."
prompt2="Write a tagline for a Sweet Company."
outputs=llm.generate([prompt1,prompt2] * 4)

print(outputs)
print(len(outputs.generations[0]), outputs.generations)

generations=[[Generation(text='\n\n"Empowering the Future with Intelligent Solutions: Revolutionizing the World with AI."', generation_info={'finish_reason': 'stop', 'logprobs': None})], [Generation(text='\n\n"Satisfy your cravings with our irresistible treats!"', generation_info={'finish_reason': 'stop', 'logprobs': None})], [Generation(text='\n\n"Unlocking the Future with Cutting-Edge AI Solutions." ', generation_info={'finish_reason': 'stop', 'logprobs': None})], [Generation(text='\n\n"Indulge in sweetness, one delectable treat at a time."', generation_info={'finish_reason': 'stop', 'logprobs': None})], [Generation(text='\n\n"Empowering the future with intelligent technology."', generation_info={'finish_reason': 'stop', 'logprobs': None})], [Generation(text='\n\n"Satisfy your sweet tooth with every bite!"', generation_info={'finish_reason': 'stop', 'logprobs': None})], [Generation(text='\n\n"Revolutionizing the Future with Intelligent Solutions: Empowering You with AI" \n', generati

In [17]:
print(outputs.generations[0])

[Generation(text='\n\n"Empowering the Future with Intelligent Solutions: Revolutionizing the World with AI."', generation_info={'finish_reason': 'stop', 'logprobs': None})]


In [18]:
for index,output in enumerate(outputs.generations):
  print(output[0].text)



"Empowering the Future with Intelligent Solutions: Revolutionizing the World with AI."


"Satisfy your cravings with our irresistible treats!"


"Unlocking the Future with Cutting-Edge AI Solutions." 


"Indulge in sweetness, one delectable treat at a time."


"Empowering the future with intelligent technology."


"Satisfy your sweet tooth with every bite!"


"Revolutionizing the Future with Intelligent Solutions: Empowering You with AI" 



"Satisfy your sweet tooth with every bite!"


## Chat Models → `gpt-3.5-turbo, gpt-4o, gpt-4o mini`

* **`AIMessage:`** AIMessage is returned from a chat model as a response to a prompt. This message represents the output of the model and consists of both
the raw output as returned by the model together standardized fields
(e.g., tool calls, usage metadata) added by the LangChain framework.

* **`HumanMessage:`** HumanMessages are messages that are passed in from a human to the model.

* **`SystemMessage:`** Message for priming AI behavior. The system message is usually passed in as the first of a sequence of input messages.

  ```
  messages = [
              SystemMessage(
                  content="You are a helpful assistant! Your name is Bob."
              ),
              HumanMessage(
                  content="What is your name?"
              )
          ]
  ```

In [19]:
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

from langchain.chat_models import ChatOpenAI

In [20]:
%%time
chat=ChatOpenAI(
    model_name="gpt-4o",
    openai_api_key=OPENAI_API_KEY,
    temperature=0,
  )
messages=[
          SystemMessage(
            content="You are an expert AI/LLM Engineer who teaches using the Richard Feynman Technique."
          ),
          HumanMessage(
            content="Explain the entire workflow of developing RAG AI Applications."
          )
        ]
print(messages)
output=chat(messages)
print(output)

[SystemMessage(content='You are an expert AI/LLM Engineer who teaches using the Richard Feynman Technique.'), HumanMessage(content='Explain the entire workflow of developing RAG AI Applications.')]


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


content="Sure! Let's break down the workflow of developing Retrieval-Augmented Generation (RAG) AI applications using the Richard Feynman Technique. This technique involves explaining concepts in simple terms, identifying gaps in understanding, and refining the explanation. Here’s a step-by-step guide:\n\n### Step 1: Understand the Basics\n\n**Retrieval-Augmented Generation (RAG)** is a hybrid approach that combines retrieval-based and generation-based methods in natural language processing (NLP). The goal is to improve the quality and relevance of generated text by incorporating external knowledge.\n\n### Step 2: Break Down the Workflow\n\n1. **Problem Definition and Data Collection**\n   - **Define the Problem**: Clearly understand the problem you are trying to solve. For example, you might want to build a chatbot that can answer questions about a specific domain.\n   - **Collect Data**: Gather a dataset that includes both the questions and the relevant documents or knowledge base th

In [21]:
print(output.content)

Sure! Let's break down the workflow of developing Retrieval-Augmented Generation (RAG) AI applications using the Richard Feynman Technique. This technique involves explaining concepts in simple terms, identifying gaps in understanding, and refining the explanation. Here’s a step-by-step guide:

### Step 1: Understand the Basics

**Retrieval-Augmented Generation (RAG)** is a hybrid approach that combines retrieval-based and generation-based methods in natural language processing (NLP). The goal is to improve the quality and relevance of generated text by incorporating external knowledge.

### Step 2: Break Down the Workflow

1. **Problem Definition and Data Collection**
   - **Define the Problem**: Clearly understand the problem you are trying to solve. For example, you might want to build a chatbot that can answer questions about a specific domain.
   - **Collect Data**: Gather a dataset that includes both the questions and the relevant documents or knowledge base that the model will u

## PromptTemplate

In [22]:
from langchain import PromptTemplate
from langchain_openai import OpenAI

In [23]:
%%time
template="""You are an expert {designation} who teaches using the Richard Feynman Technique. Keep it under {word_count} words."""

prompt=PromptTemplate(
    input_variables=["designation", "word_count"],
    template=template
)
print(prompt)

llm=OpenAI(
    model_name="gpt-3.5-turbo-instruct",
    openai_api_key=OPENAI_API_KEY,
    temperature=0.9
  )
print(llm)

output = llm.invoke(prompt.format(designation="AI Engineer", word_count=100))
print(output)

input_variables=['designation', 'word_count'] template='You are an expert {designation} who teaches using the Richard Feynman Technique. Keep it under {word_count} words.'
OpenAI
Params: {'model_name': 'gpt-3.5-turbo-instruct', 'temperature': 0.9, 'top_p': 1, 'frequency_penalty': 0, 'presence_penalty': 0, 'n': 1, 'logit_bias': {}, 'seed': None, 'logprobs': None, 'max_tokens': 256}


I am an experienced AI Engineer who uses the Richard Feynman Technique to teach complex concepts in a simple and understandable manner. This method involves breaking down complicated ideas into smaller, more manageable pieces, and explaining them using everyday language. By doing so, I ensure that my students not only understand the material, but also retain the knowledge for a longer period of time. Through this approach, I am able to bridge the gap between abstract theories and practical applications, making learning AI more accessible and engaging for my students. 
CPU times: user 125 ms, sys: 748 µs, to

## Chains

In [29]:
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains.llm import LLMChain

In [35]:
template="""You are an expert {designation} who teaches using the Richard Feynman Technique. {question}. Keep it under {word_count} words."""
prompt=PromptTemplate(
    input_variables=["designation", "question", "word_count"],
    template=template
)

# Create a runnable sequence
chain = prompt | llm
print("Chain:\n",chain)

output=chain.invoke(input={
    "designation":"AI Engineer specialized in advanced Retrieval Augmented Generation (RAG) with Large Language Models.",
    "question":"Teach about the advanced methods of RAG",
    "word_count":500
})
print("Output:\n",output)

Chain:
 first=PromptTemplate(input_variables=['designation', 'question', 'word_count'], template='You are an expert {designation} who teaches using the Richard Feynman Technique. {question}. Keep it under {word_count} words.') last=OpenAI(client=<openai.resources.completions.Completions object at 0x7b9212fcde10>, async_client=<openai.resources.completions.AsyncCompletions object at 0x7b92130baaa0>, temperature=0.9, openai_api_key=SecretStr('**********'), openai_proxy='')
Output:
 

Retrieval Augmented Generation (RAG) is a cutting-edge technique in the field of natural language processing (NLP), specifically in the realm of large language models. This method combines the power of large pre-trained language models with the ability to retrieve relevant information from a knowledge base, resulting in more accurate and comprehensive language generation.

At its core, RAG is a two-stage process. The first stage involves retrieving relevant information from a knowledge base using a retrieval

## Sequential Chains
Sequential Chains are Complex Chains.

In [49]:
from langchain_openai import OpenAI,ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, SimpleSequentialChain, SequentialChain
from langchain.schema.output_parser import StrOutputParser

In [57]:
# output parser
struct_parser=StrOutputParser()
print(struct_parser)

llm1=ChatOpenAI(
    name="gpt-3.5-turbo",
    api_key=OPENAI_API_KEY,
    temperature=0.9,
    max_tokens=1024
)

prompt1=PromptTemplate(
    input_variables=["input"],
    template="""Solve the math expression: {input}"""
)

chain1=prompt1|llm1|struct_parser
print(chain1)

llm2=ChatOpenAI(
    name="gpt-4o",
    api_key=OPENAI_API_KEY,
    temperature=0.2,
    max_tokens=1024
)

prompt2=PromptTemplate(
    input_variables=["input"],
    template="""Solve the math expression: {input}"""
)

chain2=prompt2|llm2|struct_parser
print(chain2)

sequential_chain=chain1|chain2

output=sequential_chain.invoke(input={
    "input": "5.1**7.3"
})

print(output)


first=PromptTemplate(input_variables=['input'], template='Solve the math expression: {input}') middle=[ChatOpenAI(name='gpt-3.5-turbo', client=<openai.resources.chat.completions.Completions object at 0x7b92103d3730>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x7b92103f3a00>, temperature=0.9, openai_api_key=SecretStr('**********'), openai_proxy='', max_tokens=1024)] last=StrOutputParser()
first=PromptTemplate(input_variables=['input'], template='Solve the math expression: {input}') middle=[ChatOpenAI(name='gpt-4o', client=<openai.resources.chat.completions.Completions object at 0x7b92103bda50>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x7b921039c670>, temperature=0.2, openai_api_key=SecretStr('**********'), openai_proxy='', max_tokens=1024)] last=StrOutputParser()
4074.34193072


In [60]:
output

'4074.34193072'

## Agents

In [64]:
from langchain_experimental.agents.agent_toolkits import create_python_agent
from langchain_experimental.tools import PythonREPLTool
from langchain_openai import ChatOpenAI

In [66]:
llm=ChatOpenAI(
    model="gpt-4o",
    api_key=OPENAI_API_KEY,
    temperature=0.9,
    max_tokens=1024
)
agent_executor=create_python_agent(
    llm=llm,
    tool=PythonREPLTool(),
    verbose=True
)

agent_executor.invoke(input="Calculate the square root of the factorial of 20 and display it with 4 decimal places.")



> Entering new AgentExecutor chain...


To solve this problem, I need to calculate the factorial of 20 and then find the square root of that value. Finally, I will format the result to 4 decimal places.

Action: Python_REPL
Action Input: 
```python
import math

# Calculate the factorial of 20
factorial_20 = math.factorial(20)

# Calculate the square root of the factorial
sqrt_factorial_20 = math.sqrt(factorial_20)

# Print the result with 4 decimal places
formatted_result = f"{sqrt_factorial_20:.4f}"
print(formatted_result)
```
Observation: 1559776268.6285

Thought:I now know the final answer.

Final Answer: 1559776268.6285

> Finished chain.


{'input': 'Calculate the square root of the factorial of 20 and display it with 4 decimal places.',
 'output': '1559776268.6285'}

## Pinecone Vector DB
Diving into Vector Databases.

In [70]:
PINECONE_API_KEY=userdata.get(key="PINECONE_API_KEY")

In [73]:
from pinecone import Pinecone, ServerlessSpec

In [77]:
pinecone=Pinecone(
    api_key=PINECONE_API_KEY
  )

### Pinecone Indexes

In [81]:
pinecone.list_indexes()

{'indexes': [{'deletion_protection': 'disabled',
              'dimension': 512,
              'host': 'flowise-0390e5b.svc.gcp-starter.pinecone.io',
              'metric': 'cosine',
              'name': 'flowise',
              'spec': {'pod': {'environment': 'gcp-starter',
                               'pod_type': 'starter',
                               'pods': 1,
                               'replicas': 1,
                               'shards': 1}},
              'status': {'ready': True, 'state': 'Ready'}}]}